In [1]:
''' IMPORTS '''

import pandas as pd
import numpy as np
from scipy import stats
import math
import requests
from io import BytesIO
from datetime import datetime

import nfl_data_py as nfl

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as cl
from plotly.subplots import make_subplots

from PIL import Image

from resources.tier_chart import tier_chart
from resources.plotly_theme import nfl_template
from resources.get_nfl_data import get_team_info, get_pbp_data
from resources.team_stats import get_team_data

pio.templates['nfl_template'] = nfl_template

In [149]:
''' Params / Constants '''

## Params ##
PREVIEW_WEEK = 9

AWAY_TEAM = 'IND'
HOME_TEAM = 'PIT'

## Constants ##

LEAGUE_LOGO = 'https://raw.githubusercontent.com/nflverse/nflverse-pbp/master/NFL.png'
PLOT_BUFFER = 0.1

In [150]:
''' Import Data '''

# Import
team_data = get_team_info().rename_axis(index={'team_abbr': 'team'})
pbp_data = get_pbp_data(years=[2025])

# Stats
team_offense = get_team_data(pbp_data, unit='offense')
team_offense = team_offense.merge(team_data[['team_logo_espn', 'team_wordmark']], left_index=True, right_index=True)
print(team_offense.sort_values(by='EPA / Play', ascending=False).head().to_string())

# PBP Only matchup teams
pbp_matchup_teams = pbp_data.loc[(pbp_data['week'] < PREVIEW_WEEK) &
                                 (pbp_data['posteam'].isin([HOME_TEAM,AWAY_TEAM])), :].copy()


print(team_data.shape)
print(team_data.head().to_string())
print(pbp_matchup_teams.shape)
print(pbp_matchup_teams.head().to_string())

2025 done.
Downcasting floats.
         Games  Plays   TDs  FirstDowns  RushAttempts  RushYards  RushTDs  Rush1Ds  DesignedRushPlays  DesignedRushAttempts  DesignedRushYards  QBScrambles  ScrambleYards  DesignedPassPlays  Dropbacks  PassCompletions  PassAttempts  PassYards  PassTDs  Pass1Ds  Sacks  SackYards  INTs  TFLs  Fumbles  Penalties  PenaltyYards  Penalty1Ds  Drives  Total Yards  Turnovers  PlaysAdv  PassPlays  RushPlays         EPA    RushEPA    PassEPA  Successes  RushSuccesses  PassSuccesses  EPA / Play  Rush EPA / Play  Pass EPA / Play  Success Rate  Rush Success Rate  Pass Success Rate                                     team_logo_espn                                                          team_wordmark
posteam                                                                                                                                                                                                                                                                          

# Pass / Rush Tendencies

In [151]:
## Normal Game State

# Filter 1st downs to only 1st and 10
filter_df = pbp_matchup_teams.loc[~((pbp_matchup_teams['down'] == 1) & (pbp_matchup_teams['ydstogo'] != 10)),:].copy()
filter_df['Down & Distance'] = filter_df['Down & Distance'].str.replace('1st', '1st & 10')

# Only normal snaps, Q1-3 and within 2 tds
filter_df = filter_df.loc[(filter_df['Offensive Snap']) & 
                            (~filter_df['Is Special Teams Play']) &
                            (filter_df['qtr'] <= 3) & 
                            (filter_df['score_differential'] <= 14),:]

#
filter_df['Is 1st Down'] = filter_df['down'] == 1


In [152]:
''' Overall '''

overall_gpby = filter_df.groupby(['posteam']).aggregate(
    Plays=('Offensive Snap', 'sum'),
    Pass=('pass', 'sum'),
    Rush=('rush', 'sum'),
).rename_axis(index={'posteam': 'team'})
overall_gpby['% Pass'] = overall_gpby['Pass'] / overall_gpby['Plays']
overall_gpby['% Rush'] = overall_gpby['Rush'] / overall_gpby['Plays']

print(overall_gpby)


      Plays   Pass   Rush    % Pass    % Rush
team                                         
IND     297  191.0  106.0  0.643098  0.356902
PIT     276  162.0  114.0  0.586957  0.413043


In [153]:
''' '''

first_gpby = filter_df.groupby(['posteam', 'Is 1st Down']).aggregate(
    Plays=('Offensive Snap', 'sum'),
    Pass=('pass', 'sum'),
    Rush=('rush', 'sum'),
).rename_axis(index={'posteam': 'team'})
first_gpby['% Pass'] = first_gpby['Pass'] / first_gpby['Plays']
first_gpby['% Rush'] = first_gpby['Rush'] / first_gpby['Plays']

print(first_gpby)

                  Plays   Pass  Rush    % Pass    % Rush
team Is 1st Down                                        
IND  False          159  101.0  58.0  0.635220  0.364780
     True           138   90.0  48.0  0.652174  0.347826
PIT  False          161  111.0  50.0  0.689441  0.310559
     True           115   51.0  64.0  0.443478  0.556522


In [154]:
''' Down / Distance '''

down_distance_gpby = filter_df.groupby(['posteam', 'down', 'Down & Distance']).aggregate(
                                                Plays=('Offensive Snap', 'sum'),
                                                Pass=('pass', 'sum'),
                                                Rush=('rush', 'sum'),
                                           ).rename_axis(index={'posteam': 'team'})

sub_totals_1 = down_distance_gpby.groupby(['team', 'down']).sum()
sub_totals_1['Down & Distance'] = 'Total'
sub_totals_1 = sub_totals_1.set_index('Down & Distance', append=True)

sub_totals_2 = down_distance_gpby.groupby('team').sum()
sub_totals_2['down'] = 'Total'
sub_totals_2['Down & Distance'] = 'Total'
sub_totals_2 = sub_totals_2.set_index('down', append=True)
sub_totals_2 = sub_totals_2.set_index('Down & Distance', append=True)

down_distance_gpby = pd.concat([down_distance_gpby, sub_totals_1, sub_totals_2]).sort_index()
down_distance_gpby['% Pass'] = down_distance_gpby['Pass'] / down_distance_gpby['Plays']
down_distance_gpby['% Rush'] = down_distance_gpby['Rush'] / down_distance_gpby['Plays']
# down_distance_gpby['% Viz'] = np.where(down_distance_gpby['% Pass'] >= .5, down_distance_gpby['% Pass'], -1*down_distance_gpby['% Rush'])
down_distance_gpby['Diff'] = down_distance_gpby['% Pass'] - down_distance_gpby['% Rush']

down_distance_gpby = down_distance_gpby.merge(team_data['team_logo_espn'], left_index=True, right_index=True)
# print(down_distance_gpby.to_string())


# Data
data = down_distance_gpby.loc[down_distance_gpby.index.get_level_values(2) != 'Total', :]

x = data['% Pass'].tolist()
y = data.index.get_level_values(2).tolist()
logos = data['team_logo_espn'].tolist()

y_order = ['1st & 10']
for down in ['2nd', '3rd', '4th']:
    for distance in ['Short', 'Medium', 'Long']:
        y_order.append(f'{down} & {distance}')

# Plot
dot_plot = px.scatter(
    x=x,
    y=y,
)

# Figure
fig = go.Figure()

for trace in dot_plot.data:
    fig.add_trace(trace)

# Logos
for i in range(len(logos)):
    fig.add_layout_image(
        source=logos[i],  # The loaded image
        xref="x",    # Reference x-coordinates to the x-axis
        yref="y",    # Reference y-coordinates to the y-axis
        x=x[i], # X-coordinate of the image's center
        y=y[i], # Y-coordinate of the image's center
        sizex=0.75,   # Width of the image in data units
        sizey=0.75,   # Height of the image in data units
        xanchor="center", # Anchor the image by its center horizontally
        yanchor="middle", # Anchor the image by its middle vertically
        layer="above", # Place image above other plot elements
        opacity=0.8
    )

fig.update_traces(marker=dict(opacity=0))

# x_ticks = [abs(xi) for xi in x]
fig.update_xaxes(
    title=f'More Run Heavy <--  % Pass  --> More Pass Heavy',
    tickformat='.0%',
    range=[-0.05, 1.05],
    dtick=.1,
)
fig.update_yaxes(
    # type='category',
    categoryorder="array", 
    categoryarray=y_order,
    autorange='reversed',
    showgrid=False,
)
fig.update_layout(
    template='nfl_template',
    title=f'Pass / Rush Tendencies by Down & Distance',#<br><sup>Week {PREVIEW_WEEK}: {AWAY_TEAM} vs. {HOME_TEAM}</sup>',
    margin=dict(t=60,l=75, b=75),
)

# Credits
fig.add_annotation(
    text=f'"Normal" game state: Qtrs 1-3, score within 14 pts<br>Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today().strftime("%Y-%m-%d")}',
    showarrow=False,
    xref='paper',
    yref='paper',
    y=-0.175, 
    x=1,
    align='right'
)

fig.show()


In [99]:
print(pbp_matchup_teams.head().to_string())

      play_id          game_id old_game_id home_team away_team season_type  week posteam posteam_type defteam side_of_field  yardline_100   game_date  quarter_seconds_remaining  half_seconds_remaining  game_seconds_remaining game_half  quarter_end  drive   sp  qtr  down  goal_to_go   time   yrdln  ydstogo  ydsnet                                                                                                              desc play_type  yards_gained  shotgun  no_huddle  qb_dropback  qb_kneel  qb_spike  qb_scramble pass_length pass_location  air_yards  yards_after_catch run_location run_gap field_goal_result  kick_distance extra_point_result two_point_conv_result  home_timeouts_remaining  away_timeouts_remaining  timeout timeout_team td_team td_player_name td_player_id  posteam_timeouts_remaining  defteam_timeouts_remaining  total_home_score  total_away_score  posteam_score  defteam_score  score_differential  posteam_score_post  defteam_score_post  score_differential_post  no_score_prob 

In [102]:
pass_tendencies_gpby = pbp_matchup_teams.loc[(pbp_matchup_teams['Offensive Snap']) & 
                                           (~pbp_matchup_teams['Is Special Teams Play']) &
                                           (pbp_matchup_teams['qtr'] <=3) &
                                           (pbp_matchup_teams['score_differential'] <= 10),:].groupby(['posteam', 'down', 'ydstogo']).aggregate(
                                                Plays=('Offensive Snap', 'sum'),
                                                Pass=('pass', 'sum'),
                                                Rush=('rush', 'sum'),
                                           ).rename_axis(index={'posteam': 'team'})


print(pass_tendencies_gpby.to_string())

                   Plays  Pass  Rush
team down ydstogo                   
IND  1.0  1.0          2   0.0   2.0
          2.0          1   1.0   0.0
          3.0          1   0.0   1.0
          4.0          1   0.0   1.0
          6.0          1   0.0   1.0
          7.0          1   1.0   0.0
          8.0          3   1.0   2.0
          10.0       115  73.0  42.0
          15.0         2   1.0   1.0
          20.0         2   2.0   0.0
     2.0  1.0          6   0.0   6.0
          2.0          6   1.0   5.0
          3.0          4   3.0   1.0
          4.0          5   2.0   3.0
          5.0          9   2.0   7.0
          6.0          7   3.0   4.0
          7.0          6   5.0   1.0
          8.0          6   6.0   0.0
          9.0          4   4.0   0.0
          10.0        18  17.0   1.0
          11.0         1   1.0   0.0
          12.0         2   2.0   0.0
          13.0         1   1.0   0.0
          15.0         1   1.0   0.0
          16.0         1   1.0   0.0
 

In [6]:
play_types = titans_o.groupby(['down', 'play_type']).aggregate(
    Plays=('play_type', 'size'),
    Yards=('yards_gained', 'sum')
)

print(play_types.to_string())

                Plays  Yards
down play_type              
1.0  no_play       12    0.0
     pass          98  594.0
     run           69  259.0
2.0  no_play        8    0.0
     pass          88  259.0
     run           42  166.0
3.0  no_play        6    0.0
     pass          67  209.0
     run           23  110.0
4.0  no_play        4    0.0
     pass           9   36.0
     run            1    3.0
